# QDAC-II debugging


In [1]:
from time import sleep
import pprint
pp = pprint.PrettyPrinter()
import numpy as np
from IPython.display import Image, display
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
qdac_addr = '192.168.8.153'
qdac = QDAC2.QDac2('QDAC-II', visalib='@py', address=f'TCPIP::{qdac_addr}::5025::SOCKET')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /Users/jps/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /Users/jps/.qcodes/logs/211109-10624-qcodes.log
Connected to: QDevil QDAC-II (serial:48762, firmware:5-0.9.26) in 0.32s


In [2]:
qdac.start_recording_scpi()
arrangement = qdac.arrange(
        # QDAC channels 1, 2, 3, 4 connected to sample
        gates={'plunger2': 2, 'plunger3': 3},
        # DMM external trigger connected to QDAC Output Trigger 4
        output_triggers={'dmm': 4})
pp.pprint(qdac.get_recorded_scpi_commands())

['outp:trig4:sour int1', 'outp:trig4:widt 1e-06']


In [3]:
sweep = arrangement.virtual_sweep2d(
        inner_gate='plunger2',
        inner_voltages=np.linspace(-0.2, 0.6, 5),
        outer_gate='plunger3',
        outer_voltages=np.linspace(-0.7, 0.15, 4),
        inner_step_time_s=10e-6,
        inner_step_trigger='dmm')

In [4]:
qdac.start_recording_scpi()
sweep.start()
pp.pprint(qdac.get_recorded_scpi_commands())

['sour2:dc:mark:sst 1',
 'sour2:dc:trig:sour hold',
 'sour2:volt:mode list',
 'sour2:list:volt -0.2, 0, 0.2, 0.4, 0.6, -0.2, 0, 0.2, 0.4, 0.6, -0.2, 0, '
 '0.2, 0.4, 0.6, -0.2, 0, 0.2, 0.4, 0.6',
 'sour2:list:tmod auto',
 'sour2:list:dwel 1e-05',
 'sour2:list:dir up',
 'sour2:list:coun 1',
 'sour2:dc:trig:sour bus',
 'sour2:dc:init:cont on',
 'sour2:dc:init',
 'sour2:dc:trig:sour int2',
 'sour2:dc:init:cont on',
 'sour2:dc:init',
 'sour3:dc:trig:sour hold',
 'sour3:volt:mode list',
 'sour3:list:volt -0.7, -0.7, -0.7, -0.7, -0.7, -0.416667, -0.416667, '
 '-0.416667, -0.416667, -0.416667, -0.133333, -0.133333, -0.133333, -0.133333, '
 '-0.133333, 0.15, 0.15, 0.15, 0.15, 0.15',
 'sour3:list:tmod auto',
 'sour3:list:dwel 1e-05',
 'sour3:list:dir up',
 'sour3:list:coun 1',
 'sour3:dc:trig:sour bus',
 'sour3:dc:init:cont on',
 'sour3:dc:init',
 'sour3:dc:trig:sour int2',
 'sour3:dc:init:cont on',
 'sour3:dc:init',
 'tint 2']
